In [1]:
#import pandas as pd
!wget http://www-public.tem-tsp.eu/~zhang_da/pub/dataset_tsmc2014.zip
!unzip dataset_tsmc2014.zip

--2024-05-21 10:11:03--  http://www-public.tem-tsp.eu/~zhang_da/pub/dataset_tsmc2014.zip
Resolving www-public.tem-tsp.eu (www-public.tem-tsp.eu)... 157.159.10.107, 2001:660:3203:100:1:0:80:107
Connecting to www-public.tem-tsp.eu (www-public.tem-tsp.eu)|157.159.10.107|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25546284 (24M) [application/zip]
Saving to: ‘dataset_tsmc2014.zip’

dataset_tsmc2014.zi 100%[===================>]  24.36M  6.30MB/s    in 5.8s    

2024-05-21 10:11:10 (4.17 MB/s) - ‘dataset_tsmc2014.zip’ saved [25546284/25546284]

Archive:  dataset_tsmc2014.zip
   creating: dataset_tsmc2014/
  inflating: dataset_tsmc2014/dataset_TSMC2014_NYC.txt  
  inflating: dataset_tsmc2014/dataset_TSMC2014_readme.txt  
  inflating: dataset_tsmc2014/dataset_TSMC2014_TKY.txt  


In [1]:
import pandas as pd
columns = ['User ID',
 'Venue ID',
 'Venue category ID',
 'Venue category name',
 'Latitude',
 'Longitude',
 'Timezone',
 'UTC time']

In [2]:
NUMCHECKIN = 30000

df = pd.read_csv('dataset_tsmc2014/dataset_TSMC2014_NYC.txt', sep='\t', encoding='latin-1', names=columns)
df = df.sample(NUMCHECKIN)
print(df)

        User ID                  Venue ID         Venue category ID  \
83676       106  3fd66200f964a52073e71ee3  4bf58dd8d48988d14e941735   
105044      445  4ed7215c4901772cdf8a47ba  4bf58dd8d48988d17f941735   
208219      611  4c4e136b9efabe9ab54e7b69  4bf58dd8d48988d198941735   
179948      988  4b897fd9f964a520473c32e3  4bf58dd8d48988d175941735   
92163       105  4b6755a4f964a520b4482be3  4bf58dd8d48988d1c9941735   
...         ...                       ...                       ...   
30387        80  4f1c3201e4b0c7ea68c32cb9  4bf58dd8d48988d1a9941735   
29447       155  4b00f8a6f964a5208e4122e3  4bf58dd8d48988d18e941735   
56603        53  4f1e2589e4b0475df9cd00cc  4bf58dd8d48988d103941735   
140435      663  45ca42e1f964a5207a421fe3  4bf58dd8d48988d128951735   
213910      114  4f5e82dae4b01989c438cbb6  4bf58dd8d48988d1e0931735   

              Venue category name   Latitude  Longitude  Timezone  \
83676         American Restaurant  40.765364 -73.975786      -240   
105044   

In [3]:
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()
df['Venue ID'] = le.fit_transform(df['Venue ID'])
df['Venue category ID'] = le.fit_transform(df['Venue category ID'])
df.drop(['Venue category name'],axis=1,inplace=True)
print(df)

        User ID  Venue ID  Venue category ID   Latitude  Longitude  Timezone  \
83676       106       219                117  40.765364 -73.975786      -240   
105044      445     11332                161  40.827101 -73.946214      -240   
208219      611      7568                183  40.746574 -73.995795      -300   
179948      988      5183                152  40.880434 -73.903273      -300   
92163       105      4772                224  40.893013 -73.973200      -240   
...         ...       ...                ...        ...        ...       ...   
30387        80     11695                198  40.678768 -73.995792      -240   
29447       155      3630                174  40.744279 -73.996406      -240   
56603        53     11708                  6  40.881088 -74.071136      -240   
140435      663      1002                 77  40.758788 -73.978925      -240   
213910      114     12075                245  40.758702 -73.978529      -300   

                              UTC time 

In [ ]:
from datetime import datetime

def str2date(date_str):

    parsed_date = datetime.strptime(date_str, "%a %b %d %H:%M:%S")
    print(parsed_date)

In [4]:
import math

def distance_between_coordinates(lat1, lon1, lat2, lon2):
    radius = 6371  # Radius of the Earth in km

    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Calculate the differences in latitude and longitude<
    d_lat = lat2 - lat1
    d_lon = lon2 - lon1

    # Calculate the distance using the Haversine formula
    a = math.sin(d_lat/2) * math.sin(d_lat/2) + math.cos(lat1) * math.cos(lat2) * math.sin(d_lon/2) * math.sin(d_lon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = radius * c

    return distance

In [ ]:
print(distance_between_coordinates(40.719810,-74.002581,40.735981, -74.029309))

2.8819941850086326


In [5]:
venuesdf = df[['Venue ID','Latitude','Longitude','Venue category ID']].drop_duplicates(subset=['Venue ID'])
print(venuesdf)


        Venue ID   Latitude  Longitude  Venue category ID
83676        219  40.765364 -73.975786                117
105044     11332  40.827101 -73.946214                161
208219      7568  40.746574 -73.995795                183
179948      5183  40.880434 -73.903273                152
92163       4772  40.893013 -73.973200                224
...          ...        ...        ...                ...
156417     10212  40.755886 -73.978500                 71
149270     12216  40.822902 -74.139110                289
187105     11351  40.736661 -74.029935                286
30387      11695  40.678768 -73.995792                198
29447       3630  40.744279 -73.996406                174

[13322 rows x 4 columns]


In [6]:
!pip3 install torch --index-url https://download.pytorch.org/whl/${CUDA}
import torch
!pip install torch-geometric \
  torch-sparse \
  torch-scatter \
  torch-cluster \
  -f https://pytorch-geometric.com/whl/torch-2.1.0+cu121.html

Looking in indexes: https://download.pytorch.org/whl/
Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu121.html


In [7]:
import networkx as nx
venuesdict = venuesdf.to_dict('records')

i = 0
nodeIds = {}
node_attr = []

for el in venuesdict:
  #print(el['Venue ID'])
  nodeIds[el['Venue ID']] = i
  node_attr.append(el['Venue category ID'])
  i+=1

print(len(node_attr))


13322


In [9]:
#fast spatial graph construction

import networkx as nx
venuesdict = venuesdf.to_dict('records')
#print(venuesdict)

#for el in venuesdict:
#  print((el,venuesdict[el]))
'''
G = nx.Graph()

distances = []

from tqdm import tqdm

for i in tqdm(range(len(venuesdict) - 1)):
    if not G.has_node(venuesdict[i]['Venue ID']):
        G.add_node(venuesdict[i]['Venue ID'], category=venuesdict[i]['Venue category ID'])

    for j in range(i + 1, len(venuesdict)):
        if not G.has_node(venuesdict[j]['Venue ID']):
            G.add_node(venuesdict[j]['Venue ID'], category=venuesdict[i]['Venue category ID'])

        if distance_between_coordinates(venuesdict[i]['Latitude'], venuesdict[i]['Longitude'],
                                        venuesdict[j]['Latitude'], venuesdict[j]['Longitude']) < 1:
            G.add_edge(venuesdict[i]['Venue ID'], venuesdict[j]['Venue ID'])
'''
from tqdm import tqdm
import torch
from torch_geometric.data import Data

edge_index = []
edges_attr = []

for i in tqdm(range(len(venuesdict) - 1)):

   #if venuesdict[i]['Venue ID'] not in node_attr:
   #  node_attr.append(venuesdict[i]['Venue ID'])

   for j in range(i + 1, len(venuesdict)):
    #  if venuesdict[j]['Venue ID'] not in node_attr:
    #     node_attr.append(venuesdict[j]['Venue ID'])


      if distance_between_coordinates(venuesdict[i]['Latitude'], venuesdict[i]['Longitude'],
                                    venuesdict[j]['Latitude'], venuesdict[j]['Longitude']) < 1:
        edge_index.append([nodeIds[venuesdict[i]['Venue ID']], nodeIds[venuesdict[j]['Venue ID']]])
        edges_attr.append(1)

spatG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(spatG)


100%|██████████| 13321/13321 [04:36<00:00, 48.22it/s] 


Data(x=[13322], edge_index=[2, 2165879], edge_attr=[2165879])


In [ ]:
import pickle

#spatpG = G

pickle.dump(spatG,open('spatG.pickle','wb'))

#data = pickle.load(open('spatG.pickle','rb'))

In [ ]:
#nx.draw(G)
print(spatG)

Graph with 26132 nodes and 7625550 edges


In [11]:
#spatial graph construction

print(len(venuesdf))
import networkx as nx

spatG = nx.Graph()

d = 1

# Add nodes and edges with features
for _, row1 in venuesdf.iterrows():

    if not spatG.has_node(row1['Venue ID']):
            spatG.add_node(row1['Venue ID'], category=row1['Venue category ID'])

    for _,row2 in venuesdf.iterrows():


        # Add nodes if not already in the graph


        if not spatG.has_node(row2['Venue ID']):
            spatG.add_node(row1['Venue ID'], category=row1['Venue category ID'])

        # Add edges with features

        if distance_between_coordinates(row1['Latitude'],row1['Longitude'],row2['Latitude'],row2['Longitude']) < d:

            spatG.add_edge(row1['Venue ID'],row2['Venue ID'])


print(spatG)




13416


KeyboardInterrupt: 

In [10]:

#sequantial relation graph
#build a graph that captures the sequence of actions made by the users

import networkx as nx

edge_index = []
edges_attr = []

usrloc = {}

#seqG = nx.Graph()

for _, row in df.iterrows():

  #if not seqG.has_node(row['Venue ID']):
  #  seqG.add_node(row['Venue ID'], category=row['Venue category ID'])

  if row['User ID'] not in usrloc:
    usrloc[row['User ID']] = row['Venue ID']

  else:
    #seqG.add_edge(usrloc[row['User ID']],row['Venue ID'])
    edge_index.append([nodeIds[usrloc[row['User ID']]], nodeIds[row['Venue ID']]])
    edges_attr.append(1)
    usrloc[row['User ID']] = row['Venue ID']


seqG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(seqG)






Data(x=[13322], edge_index=[2, 28917], edge_attr=[28917])


In [ ]:
import pickle


print(seqG)

pickle.dump(seqG,open('tempG.pickle','wb'))

#data = pickle.load(open('tempG.pickle','rb'))

Graph with 26132 nodes and 78480 edges


In [11]:
#build an alternative temporal graph
#two pois are connected if users go on average to them in the seme timeframe

df['localTime'] = pd.to_datetime(df['UTC time']) + pd.to_timedelta(df['Timezone'], unit = 'm')

df['localDayofWeek'] = df.localTime.dt.dayofweek
df['localHour'] = df.localTime.dt.hour






<ipython-input-11-f85843e15d2c>:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['localTime'] = pd.to_datetime(df['UTC time']) + pd.to_timedelta(df['Timezone'], unit = 'm')


In [12]:
import math
groupeddf = df.groupby(['Venue ID','Venue category ID'],as_index=False)
avghour = groupeddf['localHour'].mean()
avgday = groupeddf['localDayofWeek'].mean()

print(avghour)

timedf = pd.concat([avghour,avgday['localDayofWeek']],axis=1)



print(timedf)

hours = {}
week = {}

#week['workdays'] = []
#week['weekend'] = []


for _, row in timedf.iterrows():

  hour = math.floor(row['localHour'])
  day = math.floor(row['localDayofWeek'])

  if hour not in hours:
    #hours[hour] = [[row['Venue ID'],row['Venue category ID']]]
    hours[hour] = [[row['Venue ID'],row['Venue category ID']]]

  else:
    hours[hour].append([row['Venue ID'],row['Venue category ID']])

  #if 1 <= day <=4:
  #  week['workdays'].append([row['Venue ID'],row['Venue category ID']])
  #else:
  #  week['weekend'].append([row['Venue ID'],row['Venue category ID']])

  if day not in week:
    #hours[hour] = [[row['Venue ID'],row['Venue category ID']]]
    week[day] = [[row['Venue ID'],row['Venue category ID']]]

  else:
    week[day].append([row['Venue ID'],row['Venue category ID']])






       Venue ID  Venue category ID  localHour
0             0                153       13.0
1             1                253       23.0
2             2                 43       18.0
3             2                 56       10.5
4             3                 47       22.0
...         ...                ...        ...
13415     13317                136        2.0
13416     13318                302       16.0
13417     13319                117       17.0
13418     13320                 69        7.0
13419     13321                136        7.0

[13420 rows x 3 columns]
       Venue ID  Venue category ID  localHour  localDayofWeek
0             0                153       13.0             2.0
1             1                253       23.0             5.0
2             2                 43       18.0             4.0
3             2                 56       10.5             5.0
4             3                 47       22.0             4.5
...         ...                ...        ...     

In [13]:
#dayG = nx.Graph()
#hourG = nx.Graph()

edge_index = []
edges_attr = []

for el in hours:
   for u in hours[el]:

    #if not hourG.has_node(u[0]):
    #  hourG.add_node(u[0], category=u[1])

    for v in hours[el]:

      if u != v:

        #if not hourG.has_node(u[0]):
        #  hourG.add_node(v[0], category=v[1])

        #hourG.add_edge(u[0],v[0])
        edge_index.append([nodeIds[u[0]], nodeIds[v[0]]])

hourG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(hourG)

print(hourG)



Data(x=[13322], edge_index=[2, 10341172], edge_attr=[0])
Data(x=[13322], edge_index=[2, 10341172], edge_attr=[0])


In [14]:
edge_index = []
edges_attr = []


for el in week:
   for u in week[el]:

    #if not dayG.has_node(u[0]):
    #  dayG.add_node(u[0], category=u[1])

    for v in week[el]:

      if u != v:

        #if not dayG.has_node(u[0]):
        #  dayG.add_node(v[0], category=v[1])

        #dayG.add_edge(u[0],v[0])
        edge_index.append([nodeIds[u[0]], nodeIds[v[0]]])

dayG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(dayG)

print(dayG)


Data(x=[13322], edge_index=[2, 27208004], edge_attr=[0])
Data(x=[13322], edge_index=[2, 27208004], edge_attr=[0])
